## Predicting Subscription to Minecraft Research Newsletter
DSCI 100: Group Project

<h2> Introduction </h2>

**Players Data Set**: The players.csv data set describes data regarding individual players of the game. It has 196 observations that each have 7 variables including: 
- **experience**: Describes the players experience with the game as either "Pro", "Veteran", "Regular", or "Amateur".
- **subscribe**: Displays "TRUE" if the player is subscribed to the newsletter, and "FALSE" if they are not.
- **hashedEmail**: The hashed email (privacy safe way of displaying email) of the player. 
- **played_hours**: Amount of time (hours) player played during all sessions.
- **name**: Players first name
- **gender**: Players gender as Female, Male, Agender, Two Spirited, Non-Binary, or Perfer not to say. 
- **Age** Players age in years

**Summary Statistics:**
|Variable |Min | Max | Mean  | Q1 | Q2 | Q3 |
|---------|----|-----|------|-----|----|----|
|played_hours (hrs)|0.00|223.10|5.85|0.00|0.10|0.60|
|Age (years)|9.00|58.00|21.14|17.00|19.00|22.75|

In [38]:
library(tidyverse)
library(janitor)
library(tidymodels)
library(repr)

In [39]:
players <- read_csv("players.csv") |>
clean_names() 
head(players)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashed_email,played_hours,name,gender,age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


**Predictive Question:** Can players' experience, game-play time, and age predict whether or not an individual will subscribe to the Minecraft research newsletter in the player dataset?

<h2>Methods and Results</h2>

In [52]:
set.seed(2024)


players <- players |>
  mutate(subscribe = as_factor(subscribe),
        experience = as.factor(experience))

players <- players |>
    mutate(subscribe = subscribe, na.rm = TRUE,
           experience = experience, na.rm = TRUE,
           age = age, na.rm = TRUE,
           played_hours = played_hours, na.rm = TRUE)

#turn experience into numerical values 
players <- players |>
   mutate(experience = recode(experience,
                     "Beginner" = 1,
                     "Amateur"  = 2,
                     "Regular"  = 3,
                     "Veteran"  = 4,
                     "Pro"      = 5))

#splitting data
players_split <- initial_split(players, prop = 0.75, strata = subscribe)
players_train <- training(players_split)
players_test <- testing(players_split)

#model k-classification
knn_tune_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

#model reciep
players_recipe <- recipe(subscribe ~ experience + played_hours + age, data = players_train) |>
            step_zv(all_predictors()) |>
            step_scale(all_numeric_predictors()) |>
            step_center(all_numeric_predictors())

#folds
players_vfold <- vfold_cv(players_train, v = 5, strata = subscribe)



#values of k
k_vals <- tibble(neighbors = seq(from = 1, to = 20, by = 1))

#workflow results
knn_results <- workflow() |>
                 add_recipe(players_recipe) |>
                 add_model(knn_tune_spec) |>
                 tune_grid(resamples = players_vfold, grid = k_vals) |>
                 collect_metrics()

#determine the accuracy 
accuracies <- knn_results |>
                  filter(.metric == 'accuracy') |>
                  select(neighbors, mean)
#best K 
best_k <- accuracies |>
        arrange(desc(mean)) |>
        head(1) |>
        pull(neighbors)
best_k

#Experience has some rare categories that don't show up in all the folds so 11 for the k-value might not be correct, 
#I asked on piazza if theres a way to fix it and I'm just waiting for a response.

→ A | error:   Assigned data `orig_rows` must be compatible with existing data.
               ✖ Existing data has 28 rows.
               ✖ Assigned data has 29 rows.
               ℹ Only vectors of size 1 are recycled.
               Caused by error in `vectbl_recycle_rhs_rows()`:
               ! Can't recycle input of size 29 to size 28.

There were issues with some computations   A: x1

There were issues with some computations   A: x1





[1] 14

<h2>Discussion</h2>